In [ ]:
import pandas as pd

In [ ]:
# from load data notebook
import numpy as np
import matplotlib.pyplot as plt
import pickle

batch1 = pickle.load(open(r'./Data/batch1.pkl', 'rb'))

#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

In [ ]:
batch1['b1c15']['cycles']['10']

In [ ]:
print('cell keys:', batch1['b1c15'].keys())
print('cycle keys (1 cycle):', batch1['b1c15']['cycles']['15'].keys())
for (k, v) in batch1['b1c15']['cycles']['15'].items():
    print(k, '\t', len(v))
print('summary keys:', batch1['b1c15']['summary'].keys())
for (k, v) in batch1['b1c15']['summary'].items():
    print(k, '\t', len(v))

In [ ]:
# smaller batch test
batch_small = dict([('b1c15', batch1['b1c15']), ('b1c16', batch1['b1c16'])])
df_small = None
for i, cell in batch_small.items():
    print('processing cell', i)
    frames = [ pd.DataFrame.from_dict(cycle, orient='index').transpose() for j, cycle in cell['cycles'].items() ]
    df_c = pd.concat(frames, keys=cell['cycles'].keys())
    df_c.insert(loc=0, column='cell', value=i)
    df_c.insert(loc=1, column='charge_policy', value=cell['charge_policy'])
    # TODO other summary fields
    if df_small is None:
        df_small = df_c
    else:
        df_small = df_small.append(df_c)

df_small.reset_index(inplace=True)
df_small.rename(index=str, columns={'level_0': 'cycle', 'level_1': 'step'}, inplace=True)

In [ ]:
print(df_small.shape)
# last 50 steps of cell 16
df_small.iloc[-5:]
# first 50 steps of cell 15
# df_small.head(50)

In [ ]:
# plot dQdV's for one test cell
plt.figure(figsize=(20,10))
for i, cycle in batch1["b1c5"]["cycles"].items():
    if int(i) % 200 == 0:
        plt.plot(cycle["dQdV"][:100], label=i)
plt.title('dQdV, 1st 100 steps')
plt.legend()
plt.show()

In [ ]:
# the whole shebang
# TAKES A LONG TIME
df = None
for i, cell in batch1.items():
    print('processing cell', i)
    frames = [ pd.DataFrame.from_dict(cycle, orient='index').transpose() for j, cycle in cell['cycles'].items() ]
    df_c = pd.concat(frames, keys=cell['cycles'].keys())
    df_c.insert(loc=0, column='cell', value=i)
    df_c.insert(loc=1, column='charge_policy', value=cell['charge_policy'])
    # TODO other summary fields
    if df is None:
        df = df_c
    else:
        df = df.append(df_c)

df.reset_index(inplace=True)
df.rename(index=str, columns={'level_0': 'cycle', 'level_1': 'step'})

In [ ]:
print(df.shape)  # (36468361, 11)
df.head()

In [ ]:
df.charge_policy.value_counts()